In [362]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter
import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio import plot
import folium

In [137]:
# Init Sentinel API
username = 'cryovision' 
password = 'baw.YFV8rfa8nva9mfg' 
api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

In [138]:
SHAPEFILE = 'pingo_distribution_siberia_ggrosse_bjones/pingo_distribution_siberia_ggrosse_bjones.shp'

In [224]:
pingo = gpd.read_file(SHAPEFILE)

In [551]:
pingo = gpd.read_file(SHAPEFILE)
pingo = pingo.to_crs('epsg:4326')

In [569]:
geojson_to_wkt(read_geojson('pingo.geojson')['features'][0])

'POINT(-174.3428 65.5855)'

In [572]:
# Import Shapefile

pingo_number = 1
pingo = gpd.read_file(SHAPEFILE)
pingo = pingo.to_crs('epsg:4326')
pingo.to_file('pingo.geojson', driver='GeoJSON')
geojson = read_geojson('pingo.geojson')['features'][pingo_number]
footprint = geojson_to_wkt(geojson)

In [575]:
# Get Images from api

images = api.query(footprint,
                        platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10),
                     limit=100)
dataframe = api.to_geodataframe(images)
dataframe_sorted = dataframe.sort_values(['ingestiondate','cloudcoverpercentage'], ascending=[False,True])

In [308]:
# # Download Image Data set using Id
path_filter = make_path_filter("*_B0[234]_60m.jp2")
download_data = api.download(dataframe_sorted.index[0],nodefilter=path_filter)

In [365]:
main_path = download_data['node_path'][2:]
image_paths = list(download_data['nodes'].keys())[1:]
b2 = rio.open(main_path+list(download_data['nodes'].keys())[1:][0][1:])
b3 = rio.open(main_path+list(download_data['nodes'].keys())[2:][0][1:])
b4 = rio.open(main_path+list(download_data['nodes'].keys())[3:][0][1:])

In [522]:
pingo_proj = pingo.to_crs(b4.crs.data.get('init'))
with rio.open('RGB.tiff','w',driver='GTiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),3) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),1) 
    rgb.close()



In [523]:
with rio.open("RGB.tiff") as src:
    out_image, out_transform = rasterio.mask.mask(src, pingo_proj.geometry.buffer(40000.0),crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rasterio.open("RGB_masked.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [508]:
src=rasterio.open('RGB.tif')
data = src.read([1,2,3])
norm = (data * (255 / np.max(data))).astype(np.uint8)